In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
housing = pd.read_csv('/mnt/New_Volume/Work_From_Home/Data/housing.csv')
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
housing.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [4]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')

In [7]:
scaler = StandardScaler()
encoder = OneHotEncoder()

In [43]:
class converter(BaseEstimator, TransformerMixin):
    def fit(self,x):
        data_num = np.array(x)[:,:8]
        # data_cat = np.array(x)[:,-1]
        data_num = imputer.fit(data_num)
        # data_num = scaler.fit(data_num)
        
        # data_cat = encoder.fit(data_cat.reshape(-1,1))
        
        # data = np.c_([data_num,data_cat])
        return self
        
    def transform(self, x):
        data_num = np.array(x)[:,:8]
        data_cat = np.array(x)[:,-1]
        data_num = imputer.transform(data_num)
        # data_num = scaler.transform(data_num)
        
        # data_cat = encoder.transform(data_cat.reshape(-1,1)).toarray()
        
        data = np.c_([data_num,data_cat])
        return data
        
class encode (BaseEstimator,TransformerMixin):
    def fit(self, x, y =None):
        
        data_cat = x[:-1]
        data_cat = encoder.fit(data_cat.reshape(-1,1))
        return self
    def transform(self, x):
        data_num = np.array(x)[:,:8]
        data_cat = x[:-1]
        data_cat = encoder.transform(data_cat)
        data = np.c_[data_num,data_cat]
        return data


In [44]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()

In [45]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([('convert', converter),('scale', encode),('linear Regression',lin_reg)])

In [25]:
x = housing.drop('median_house_value',axis = 1)
y = housing['median_house_value']

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state=42)

In [32]:
data_num = x_train.select_dtypes(exclude = 'object')
data_num = imputer.fit_transform(data_num)
data_num = scaler.fit_transform(data_num)

In [34]:
data_cat = x_train.select_dtypes(include = 'object')
data_cat = encoder.fit_transform(np.array(data_cat).reshape(-1,1)).toarray()

In [36]:
x_train = np.c_[data_num, data_cat]

In [71]:
lin_reg.fit(x_train,y_train)

LinearRegression()

In [105]:
def transformer(X):
    x = np.array([X])
    data_num = x[:,:]
    print('data_num',data_num)
    data_cat = x[:,-1]
    data_num = imputer.transform(data_num)
    print(data_num)
    data_num = scaler.transform(data_num)
    data_cat = encoder.transform(data_cat.reshape(-1,1)).toarray()
    return np.c_[data_num,data_cat]

In [106]:
transformer(test_data)

data_num [[[-119.01 36.06 25.0 ... 359.0 1.6812 'INLAND']
  [-119.46 35.14 30.0 ... 584.0 2.5313 'INLAND']
  [-122.44 37.8 52.0 ... 963.0 3.4801 'NEAR BAY']
  ...
  [-122.05 37.31 25.0 ... 568.0 9.2298 '<1H OCEAN']
  [-119.76 36.77 36.0 ... 474.0 2.785 'INLAND']
  [-118.37 34.22 17.0 ... 448.0 3.5521 '<1H OCEAN']]]


ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'INLAND'

In [99]:
test_data = np.array(x_test)
# test_data[:,:8]
# res = lin_reg.predict(transformer(test_data))

In [94]:
pd.set_option('display.max_rows', 301)
x_test.iloc[:300,:]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,INLAND
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,INLAND
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,NEAR BAY
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,<1H OCEAN
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,NEAR OCEAN
13311,-117.61,34.08,12.0,4427.0,NaN,2400.0,843.0,4.7147,INLAND
7113,-118.02,33.89,36.0,1375.0,NaN,670.0,221.0,5.0839,<1H OCEAN
7668,-118.08,33.92,38.0,1335.0,NaN,1011.0,269.0,3.6908,<1H OCEAN
18246,-122.08,37.39,4.0,2292.0,NaN,1050.0,584.0,4.8036,NEAR BAY
5723,-118.23,34.18,45.0,2332.0,NaN,943.0,339.0,8.1132,<1H OCEAN
